<a href="https://colab.research.google.com/github/Rt247/Not_NLP_CW/blob/master/NLP_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>